# Basic Ranked Retrieval (RRI)

In [131]:
import pickle
import timeit
import numpy as np
import pandas as pd
from xml.dom import minidom
from xml.etree import cElementTree as ElementTree
import os
import nltk
import ssl
import math
import csv

try:
     _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Reading of Inverted Index file

In [132]:
def indexReader():
    """
    Reads the inverted index created in the distributed_inverted_index.ipynb notebook
    The path from where it reads the file is docs/inverted_index.pkl
    :return: the inverted index, represented in a dictionary where the keys are the terms
    and the values is also a dictionary that contains the frecuency of documents
    that contain the term, and the posting. The posting is an array itself
    that contains the document id, and the term's frequency in that document.
    {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    """
    with open('docs/inverted_index.pkl', 'rb') as index:
        return pickle.load(index)

invertedIndex = indexReader()


## Documents and Queries Tokenization

In [133]:
## Imports
def documentReader():
    """
    This method reads the documents
    :return: Dictionary of documents {dXXX: content of document dXXX}
    """
    documents_path = os.path.join(os.getcwd(), 'docs/docs-raw-texts')
    documentos = {}
    documents_paths = os.listdir(documents_path)
    documents_paths.sort()
    #print(documents_paths)
    for filename in documents_paths:
        file_path = os.path.join(documents_path, filename)
        #print(filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        title = xmldoc.getElementsByTagName('fileDesc')[0].attributes['title'].value
        data = next(ElementTree.parse(file_path).iter('raw')).text
        documentos[id] = (title + ' ' + data).replace(u'\xa0', u' ').replace('\n', ' ')
    return documentos

documentos = documentReader()
print(list(documentos.items())[0])

('d001', 'William Beaumont and the Human Digestion William Beaumont and the Human Digestion.  William Beaumont: Physiology of digestion Image Source.  On November 21, 1785, US-American surgeon William Beaumont was born. He became best known as “Father of Gastric Physiology” following his research on human digestion. William Beaumont was born in Lebanon, Connecticut and became a physician. He served as a surgeon’s mate in the Army during the War of 1812. He opened a private practice in Plattsburgh, New York, but rejoined the Army as a surgeon in 1819. Beaumont was stationed at Fort Mackinac on Mackinac Island in Michigan in the early 1820s when it existed to protect the interests of the American Fur Company. The fort became the refuge for a wounded 19-year-old French-Canadian fur trader named Alexis St. Martin when a shotgun went off by accident in the American Fur Company store at close range June 6th, 1822. St. Martin’s wound was quite serious because his stomach was perforated and se

In [134]:
def queries_reader():
    """
    This method reads the queries
    :return: Dictionary of queries {qYY: content of query qYY}
    """
    queries_path = os.path.join(os.getcwd(), 'docs/queries-raw-texts')
    queries = {}
    queries_paths = os.listdir(queries_path)
    queries_paths.sort()
    #print(documents_paths)
    queries_index = []
    for filename in queries_paths:
        file_path = os.path.join(queries_path, filename)
        #print(filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        query = next(ElementTree.parse(file_path).iter('raw')).text
        queries[id] = query.replace(u'\xa0', u' ').replace('\n', ' ')
        queries_index.append(id)
    return queries, queries_index

queries, queries_index = queries_reader()
print(list(queries.items())[0])

('q01', 'Fabrication of music instruments')


In [135]:
def queries_tokenization(queries):
    """
    Queries tokenization using nltk. The returned dictionary,
    :param queries in a dictionary, {qYY: content of query qYY}
    :return: Dictionary of tokenized queries {qYY: ["term1","term2",...,"termn"]}
    """
    nltk_stop_words_en = set(nltk.corpus.stopwords.words("english"))
    wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
    #print("items", queries.items())
    tokenized_queries = {}
    for key,doc in queries.items():
        word_tok = nltk.word_tokenize(doc)
        word_tok_sw = [token for token in word_tok if (token.isalnum()) and (token not in nltk_stop_words_en)
                       ]
        nltk_lemmaList = [wordnet_lemmatizer.lemmatize(word) for word in word_tok_sw]
        #print(nltk_lemmaList)
        tokenized_queries[key] = nltk_lemmaList

    return tokenized_queries



tokenized_queries = queries_tokenization(queries)
print(list(tokenized_queries.items())[0])

('q01', ['Fabrication', 'music', 'instrument'])


## Score Function

In [136]:
def score(query, document_id, invertedIndex, N):
    """
    Computes to score for a query, document pair
    :param query: tokenized query in a document,  {qYY: ["term1","term2",...,"termn"]}
    :param document_id: id of the document in "dxxx" notation
    :param invertedIndex: inverted index in a dictionary, {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    :param N: the amount of documents
    :return: the score of the query (decimal)
    """
    score = 0

    tokens = query[1]
    for token in tokens:
        token_in_index = invertedIndex.get(token, "unknown_token")
        if token_in_index != "unknown_token":
            token_posting = token_in_index["posting"]
            tf = 0
            tf_w = 0
            for doc_freq_pair in token_posting:
                if doc_freq_pair[0] == document_id:
                    tf = doc_freq_pair[1]
                    tf_w = math.log10(1 + tf)
                    break
            df = token_in_index["freq"]
            idf = math.log10( N / df )
            score += tf_w*idf
    return  score


query1 = list(tokenized_queries.items())[3]

scoreQ1 = score(query1, 'd001', invertedIndex, len(documentos))
print(scoreQ1)


0.0


## Score function alternative

In [137]:
def basic_ranked_retrieval(queries,invertedIndex,documents,N):
    """
    Computes the score for all the queries
    :param queries: tokenized queries in a dictorionary {qYY: ["term1","term2",...,"termn"]}
    :param invertedIndex: inverted index in a dictionary {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    :param documents: tokenized documents in a dictionary {dXXX: content of document dXXX}
    :param N: total number of documents
    :return: a dictionary with the scores for all the queries for each document
    {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    """

    scores= {}
    query_scores_template = {}
    for key, doc in documents.items():
        query_scores_template[key] = 0

    for query,tokens in queries.items():
        query_scores = query_scores_template.copy()
        for token in tokens:
            if token in invertedIndex:
                df = invertedIndex[token]["freq"]
                idf = math.log10( N / df )
                for docs in invertedIndex[token]["posting"]:
                    tf = docs[1]
                    tf_w = math.log10(1 + tf)
                    #if docs[0] not in query_scores:
                    #    query_scores[docs[0]] = 0
                    docId = "d{0:0=3d}".format(docs[0])
                    query_scores[docId] += tf_w*idf
        clean_query_scores = { k : v for k,v in query_scores.items() if v != 0}
        clean_query_scores = dict(sorted(clean_query_scores.items(), key=lambda item: item[1], reverse=True))
        scores[query] = clean_query_scores#query_scores

    return scores

start_time = timeit.default_timer()
print("The start time is :",start_time)
RRI = basic_ranked_retrieval(tokenized_queries,invertedIndex, documentos, len(documentos))
stop_time = timeit.default_timer()
print("The stop time is :", stop_time)
print("The time difference is :", stop_time - start_time, 'seconds')

#print(scores)

The start time is : 8506.557397442
The stop time is : 8506.572125416
The time difference is : 0.014727973999470123 seconds


## Results file

In [138]:
def writeScoreFile(RRI):
    """
    Writes the RRI-queries_results.tsv that contains the score of each query
    :param RRI: Dictionary with the scores, {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    :return: none
    """
    file_path = os.path.join(os.getcwd(), 'docs/answer_files/RRI-queries_results.tsv')
    with open(file_path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for query_id,scores in RRI.items():
            scores_list = ""
            for doc,score in scores.items():
                scores_list+= doc +":"+str(round(score,4))+","
            tsv_writer.writerow([query_id,scores_list[:-1]])


writeScoreFile(RRI)
print("Archivo escrito")


Archivo escrito


In [139]:
def RRI_simplifier(RRI):
    """
    Simplifica los resultados, eliminando los puntajes.
    :param RRI: Dictionary with scores {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    :return: {'qXX':['dXXX','dYYY',...]}
    """
    result = {}
    for qid,scores in RRI.items():
        docs = []
        for k,v in scores.items():
            docs.append(k)
        result[qid]=docs
    return result


results = RRI_simplifier(RRI)
results['q01'][:5]


['d016', 'd254', 'd186', 'd153', 'd209']

## Evaluation

In [140]:
def read_judgemnts_file():
    """
    Lee el archivo de relevancia de los jueces
    :return: Diccionario con pares key: value, donde el key es el id de cada query y el value
    es otro doccionario con las ids de los docs relevantes para esa query ordenados en forma creciente.
    """
    document_path = os.path.join(os.getcwd(), 'docs/relevance-judgments.tsv')
    tsv_file = open(document_path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    relevance = {}
    for row in read_tsv:
        documents = row[1].split(',')
        query_relevance = {pair.split(':')[0] : pair.split(':')[1] for pair in documents }
        query_relevance = dict(sorted(query_relevance.items(), key=lambda item: item[0]))
        relevance[row[0]] = query_relevance
    return relevance


relevance = read_judgemnts_file()
print(relevance['q01'])

{'d016': '5', 'd186': '4', 'd254': '5'}


### Maping of results to numeric and binary vectors

In [141]:
def make_binary_result(results, relevant_res):
    """
    Este método toma los resultados crudos obtenidos para las queries (Para cada query la lista de documentos ordenaos
    por relevancia), devuelve 3 representaciones de estos resultados. La primera es la representacion binaria at K.
    Que es del mismo tamaño que el número de documentos relevantes. La segunda es esta misma lista pero con la escala
    dada por el archivo de evaluación. La tercera está destinada al cálculo del MAP, tiene la representación binaria
    hasta que salgan todos los documentos relevantes o simplemente de todos los documentos, además en su segundo
    componente tiene el número de documentos relevantes que deberían salir en los resultados según el archivo de
    evaluación.
    :param results: Diccionario con resultados crudos de cada query. Ej: {'q01': ['d254', 'd016', 'd153', ...]}
    :param relevant_res: Las 3 representaciones antes mencionadas
    :return:
    """
    bin_relevant = {}
    rel_scale_repr = {}
    map_relevant_docs = {}
    for query, relevant_docs in relevant_res.items():
        bin_repr = []
        scaled_repr = []
        map_repr = []
        M = len(relevant_docs)
        for doc_id, rel_scale in relevant_docs.items():
            bin = 1 if doc_id in results[query][:M] else 0
            bin_repr.append(bin)
            scaled_repr.append(bin * int(rel_scale))
        i = 0
        for doc_id in results[query]:
            if i < M:
                map_bin = 1 if doc_id in relevant_res[query] else 0
                i += map_bin
                map_repr.append(map_bin)
        bin_relevant[query] = bin_repr
        rel_scale_repr[query] = scaled_repr
        map_relevant_docs[query] = [map_repr, M]
    return bin_relevant, rel_scale_repr, map_relevant_docs

bin_results, scaled_results, map_relevant_docs = make_binary_result(results, relevance)
print(bin_results['q01'])
print(scaled_results['q01'])

[1, 1, 1]
[5, 4, 5]


In [142]:
print('Primeros 5 documentos devueltos como relevantes para q01: \n', results['q01'][:5])
print('Documentos relevantes para q01 según jueces: \n' , relevance['q01'])
print('Representación binaria de q01, hasta el último doc relevante: \n' ,map_relevant_docs['q01'])

Primeros 5 documentos devueltos como relevantes para q01: 
 ['d016', 'd254', 'd186', 'd153', 'd209']
Documentos relevantes para q01 según jueces: 
 {'d016': '5', 'd186': '4', 'd254': '5'}
Representación binaria de q01, hasta el último doc relevante: 
 [[1, 1, 1], 3]


### Definition of IR metrics functions

In [143]:
def precision_at_k(relevance: list, k: int):
    """
    DocString
    :return: Nothing
    """
    if k == 0:
        return 0
    l = np.array(relevance[:k]).sum()/k
    return l

def recall_at_k(relevance: list, nr_relevant: int, k: int):
    """
    DocString
    :return: Nothing
    """
    l = np.array(relevance[:k]).sum()/nr_relevant
    return l

def average_precision(relevance):
    """
    DocString
    :return: Nothing
    """

    length = len(relevance[0])
    sum = 0
    for i in range(length):
        if relevance[0][i]:
            sum += precision_at_k(relevance[0], i+1)
    if np.array(relevance[0]).sum()==0:
        return 0
    else:
        return sum / relevance[1]

def mean_avg_precision(l):
    """
    DocString
    :return: Nothing
    """
    mean = np.array([ average_precision(lista) for lista in l]).mean()
    return mean

mean_avg_precision([[[0, 0, 0, 0, 0, 0, 1], 1], [[0, 0, 0, 1, 1], 2], [[0, 1, 0, 1, 1, 1, 1], 5]])

0.35468253968253965

In [144]:
def dcg_at_k(relevance, k: int):
    """
    Calcula el DCG at K de un vector binario representando los resultados relevantes para una query.
    :param relevance: Vector binario
    :return: DCG at K de nuestra query
    """

    sum = 0
    i =  0
    for rel_i in relevance[: k]:
        i+= 1
        sum += rel_i/np.log2(max(i, 2))

    return sum

dcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

def ndcg_at_k(relevance, k):
    """
    Calcula el ndcg at k de un vector binario
    :return: NDCG at K.
    """
    rel_sorted = sorted(relevance, reverse=True)
    max = dcg_at_k(rel_sorted, k)
    real = dcg_at_k(relevance, k)

    return real/ max if max != 0 else 0


ndcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

0.7424602308163405

In [145]:
print(recall_at_k(bin_results['q01'], 3, 3))


1.0


## Compute Evaluation Metrics for each query

In [146]:
def evaluation_metric(bin_queries, query_index, scaled_results):
    """

    :param bin_queries: Diccionario con valores {query Key: vector}, donde el vector corresponde a una lista
    con la representación binaria de un de los resultados encontrados para una query con relación a los dados
    en el archivo de evaluación. Ej, para q01, los relevantes son: d186,d254,d016. El RRDV devuelve d254, d016,
    d153. Por ende, la representación binaria de q01, en el orden del archivo de evaluación es: [0, 1, 1]
    :param query_index: Lista con los ids de las queries. ['qo1', 'qo2', ...]
    :param scaled_results: Representación escalada de los resultados de las queries usando la escala dada en el
    archivo de evaluación. Ej, q01 pasa de [0, 1, 1] a [0, 5, 5]
    :return: Un dataframe con el cálculo del P@M, r@M y NDCG@M para cada query
    """
    COLUMNS = ['P@M', 'R@M', 'DCG@M', 'NDCG@M']
    records = []
    for query, bin_vec in bin_queries.items():
        scaled = scaled_results[query]
        M = len(bin_vec)
        pm = precision_at_k(bin_vec, M)
        rm = recall_at_k(bin_vec, M, M)
        dcg = dcg_at_k(scaled, M)
        ndcg = ndcg_at_k(scaled, M)
        records.append([pm, rm,dcg, ndcg])

    return pd.DataFrame.from_records(records, index=query_index, columns=COLUMNS)

metrics = evaluation_metric(bin_results, queries_index, scaled_results)
metrics

,P@M,R@M,DCG@M,NDCG@M
q01,1.000000,1.000000,12.154649,0.970530
q02,0.000000,0.000000,0.000000,0.000000
q03,0.000000,0.000000,0.000000,0.000000
q04,0.000000,0.000000,0.000000,0.000000
q06,0.000000,0.000000,0.000000,0.000000
q07,0.000000,0.000000,0.000000,0.000000
q08,0.666667,0.666667,14.386539,0.835863
q09,0.000000,0.000000,0.000000,0.000000
q10,0.125000,0.125000,2.000000,1.000000
q12,0.000000,0.000000,0.000000,0.000000


### MAP

In [147]:
def overall_map(map_relevant_docs):
    """
    Función que calcula el MAP de los resultados de las queries.
    :param map_relevant_docs: Vector binario de las queries asegurandose de que aparezcan todos los documentos relevantes
    :return: El Mean average precision de los resultados de las queries.
    """
    matrix = [vector for key, vector in map_relevant_docs.items() ]
    return mean_avg_precision(matrix)

print(f'MAP resultante de todas las queries: {overall_map(map_relevant_docs)}')


MAP resultante de todas las queries: 0.10064463676195876
